# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database into DataFrame df
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('DisasterData', con=engine)

In [3]:
# split Dataframe df into X and y
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):

    # tokenize text and instantiate lemmatizer
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    
    # remove punctuaction
    tokens = [token for token in tokens if token.isalpha()]

    # create clean tokens
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# build pipeline for the text transformation and for estimator
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# split data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
%%time
# train data
pipeline.fit(X_train, y_train)

CPU times: user 2min 37s, sys: 7.35 s, total: 2min 45s
Wall time: 2min 45s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [8]:
y_test.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# predict responses for basic model
Y_pred = pipeline.predict(X_test)

In [10]:
# print classification_report
print(classification_report(y_test, Y_pred, target_names = y_test.columns, digits = 2))

                        precision    recall  f1-score   support

               related       0.87      0.91      0.89      5105
               request       0.82      0.44      0.57      1140
                 offer       0.00      0.00      0.00        33
           aid_related       0.75      0.60      0.66      2739
          medical_help       0.60      0.07      0.12       546
      medical_products       0.88      0.11      0.19       355
     search_and_rescue       0.54      0.04      0.08       173
              security       0.00      0.00      0.00       123
              military       0.69      0.10      0.17       226
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.29      0.43       414
                  food       0.80      0.60      0.69       727
               shelter       0.83      0.30      0.44       592
              clothing       0.69      0.12      0.20        95
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# build pipeline for the text transformation and for estimator
cv_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [12]:
MultiOutputClassifier(RandomForestClassifier()).get_params()

{'estimator__bootstrap': True,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 10,
 'estimator__n_jobs': 1,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
         

In [13]:
# reduced hyperparameter tuning and cross validation due to runtime
parameters = {
    'clf__estimator__n_estimators': [4, 6, 9],
    'clf__estimator__min_samples_split': [2, 3, 5],
}
    

cv_forest = GridSearchCV(cv_pipeline, param_grid = parameters, cv = 2, verbose = 2, n_jobs = 4)

In [14]:
%%time
# train improved model
cv_forest.fit(X_train, y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=4 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=4 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=6 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=6 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=4, total= 7.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=9 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=4, total= 7.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=9 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=6, total= 8.2min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=4 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=6, total= 8.2min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estima

[Parallel(n_jobs=4)]: Done  18 out of  18 | elapsed: 53.2min finished


CPU times: user 2min 13s, sys: 8.24 s, total: 2min 21s
Wall time: 55min 28s


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__n_estimators': [4, 6, 9], 'clf__estimator__min_samples_split': [2, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [15]:
# display the best performing parameters
cv_forest.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 6}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# predict responses for improved model
Y_pred_cv = cv_forest.predict(X_test)

In [17]:
# print classification_report
print(classification_report(y_test, Y_pred_cv, target_names = y_test.columns, digits = 2))

                        precision    recall  f1-score   support

               related       0.87      0.89      0.88      5105
               request       0.79      0.39      0.52      1140
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.55      0.63      2739
          medical_help       0.60      0.09      0.15       546
      medical_products       0.70      0.08      0.14       355
     search_and_rescue       0.53      0.05      0.09       173
              security       0.00      0.00      0.00       123
              military       0.56      0.09      0.15       226
           child_alone       0.00      0.00      0.00         0
                 water       0.81      0.32      0.46       414
                  food       0.82      0.43      0.56       727
               shelter       0.79      0.26      0.39       592
              clothing       1.00      0.05      0.10        95
                 money       0.78      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

I'm going to use the KNeighborsClassifier like in my previous submission and see how well it performs in this case.

In [18]:
# using KNeighborsClassifier
pipeline_knn = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [19]:
# train KNeighborsClassifier model
pipeline_knn.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [20]:
# predict responses for KNeighborsClassifier model
Y_pred_knn = pipeline_knn.predict(X_test)

In [21]:
# print classification_report
print(classification_report(y_test, Y_pred_knn, target_names = y_test.columns, digits = 2))

                        precision    recall  f1-score   support

               related       0.80      0.99      0.88      5105
               request       0.75      0.09      0.16      1140
                 offer       0.00      0.00      0.00        33
           aid_related       0.78      0.05      0.09      2739
          medical_help       0.00      0.00      0.00       546
      medical_products       0.50      0.00      0.01       355
     search_and_rescue       0.00      0.00      0.00       173
              security       0.00      0.00      0.00       123
              military       0.00      0.00      0.00       226
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.05      0.09       414
                  food       0.75      0.07      0.12       727
               shelter       0.84      0.04      0.07       592
              clothing       0.50      0.02      0.04        95
                 money       0.75      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [22]:
pickle_out = open('model.pkl','wb')
pickle.dump(cv_forest, pickle_out)
pickle_out.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.